In [38]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [25]:
df = pd.read_csv('TMDB_movie_dataset_v11.csv')
df = df.dropna(subset=["title", 'overview'])
df = df.drop_duplicates()

df.head(10)


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,False,/en971MEXui9diirXlogOrPKmsEn.jpg,...,Deadpool,The origin story of former Special Forces oper...,72.735,/zq8Cl3PNIDGU3iWNRoc5nEZ6pCe.jpg,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,...,Avengers: Infinity War,As the Avengers and their allies have continue...,154.340,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,An entire universe. Once and for all.,"Adventure, Action, Science Fiction",Marvel Studios,United States of America,"English, Xhosa","sacrifice, magic, superhero, based on comic, s..."
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,Fight Club,A ticking-time-bomb insomniac and a slippery s...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o..."
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,False,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,...,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,/r7vmZjiyZw9rpJMQJdXpjgiCOk9.jpg,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacec

In [33]:
df = df.loc[df['adult'] == False] # Filter out adult movies
df = df.loc[df['status'] == 'Released'] # Filter out unreleased movies

df = df[['id', 'title', 'overview', 'keywords', 'genres', 'popularity']]
df

,id,title,overview,keywords,genres,popularity
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...","rescue, mission, dream, airplane, paris, franc...","Action, Science Fiction, Adventure",83.952
1,157336,Interstellar,The adventures of a group of explorers who mak...,"rescue, future, spacecraft, race against time,...","Adventure, Drama, Science Fiction",140.241
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"joker, sadism, chaos, secret identity, crime f...","Drama, Action, Crime, Thriller",130.643
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","future, society, culture clash, space travel, ...","Action, Adventure, Fantasy, Science Fiction",79.932
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,"new york city, superhero, shield, based on com...","Science Fiction, Action, Adventure",98.082
...,...,...,...,...,...,...
1274080,839742,The Silent Battle: Mental Health & Addiction i...,Nile Wilson shares his story of mental health ...,NaN,Documentary,0.600
1274081,273967,MatchMakers,An exploration of matchmaking and the pursuit ...,NaN,Documentary,0.600
1274082,839744,Berlin: Hasenheide,Documentary about the social microcosm of Hase...,"berlin, germany, park",Documentary,0.600
1274083,839745,Puncture,When one of his crew goes missing with a bag f...,NaN,"Action, Thriller",0.600


In [34]:
dotenv_path = ('../.env')
load_dotenv(dotenv_path)

API_READ_ACCESS = os.environ.get("API_READ_ACCESS")
API_KEY = os.environ.get("API_KEY")
URL= "https://api.themoviedb.org/3"

In [41]:
def get_cast_crew(movie_id: int):
    """
    Fetch cast details from The Movie Database API.
    """
    global success_count
    
    
    params = { # Parameters for the API request
        
        'api_key': API_KEY
    }

    try:
        response = requests.get(f"{URL}/movie/{movie_id}/credits", params=params) # Get the cast details
        response.raise_for_status() # Raise an error for bad responses

        data = response.json()

        count = 0 # Initialize a counter for cast members
        cast = [] # Initialize an empty list to store cast members

        for member in data['cast']:
            if count < 4:    # Limit to the first 4 cast members    
                if member['known_for_department'] == 'Acting':
                    cast.append(member['name']) # Add the cast member's name to the list
                    count += 1 # Increment the counter
    
        if data['crew']:
                try:
                    cast.append(data['crew'][0]['name']) # Add the first crew member's name to the cast list
                except IndexError:
                    print(f"No crew members found for movie ID {movie_id}.")

        success_count += 1
        print(f"Successfully fetched cast and crew for movie ID {movie_id}. Total successful: {success_count}")

        return cast # Return the list of cast and crew members
    
    except requests.exceptions.HTTPError as err:
        print(f'Error fetching cast and crew for movie ID {movie_id}:', err)
        return []
    


        

In [ ]:
mask = df['popularity'] > 5     # Create a mask for movies with popularity greater than 5
# Initialize the cast column with empty strings
df['cast_crew'] = " " 

# Counter for successful fetches
success_count = 0

# Function to fetch and count cast members (applied to each row)
df.loc[mask, 'cast_crew'] = df.loc[mask, 'id'].apply(get_cast_crew)

Successfully fetched cast and crew for movie ID 27205. Total successful: 1
Successfully fetched cast and crew for movie ID 157336. Total successful: 2
Successfully fetched cast and crew for movie ID 155. Total successful: 3
Successfully fetched cast and crew for movie ID 19995. Total successful: 4
Successfully fetched cast and crew for movie ID 24428. Total successful: 5
Successfully fetched cast and crew for movie ID 293660. Total successful: 6
Successfully fetched cast and crew for movie ID 299536. Total successful: 7
Successfully fetched cast and crew for movie ID 550. Total successful: 8
Successfully fetched cast and crew for movie ID 118340. Total successful: 9
Successfully fetched cast and crew for movie ID 680. Total successful: 10
Successfully fetched cast and crew for movie ID 13. Total successful: 11
Successfully fetched cast and crew for movie ID 671. Total successful: 12
Successfully fetched cast and crew for movie ID 1726. Total successful: 13
Successfully fetched cast and

In [49]:
df = df.fillna(' ') # Fill NaN values with empty strings
df

,id,title,overview,keywords,genres,popularity,cast_crew
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...","rescue, mission, dream, airplane, paris, franc...","Action, Science Fiction, Adventure",83.952,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Ken ..."
1,157336,Interstellar,The adventures of a group of explorers who mak...,"rescue, future, spacecraft, race against time,...","Adventure, Drama, Science Fiction",140.241,"[Matthew McConaughey, Anne Hathaway, Michael C..."
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"joker, sadism, chaos, secret identity, crime f...","Drama, Action, Crime, Thriller",130.643,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","future, society, culture clash, space travel, ...","Action, Adventure, Fantasy, Science Fiction",79.932,"[Sam Worthington, Zoe Saldaña, Sigourney Weave..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,"new york city, superhero, shield, based on com...","Science Fiction, Action, Adventure",98.082,"[Robert Downey Jr., Chris Evans, Mark Ruffalo,..."
...,...,...,...,...,...,...,...
1274080,839742,The Silent Battle: Mental Health & Addiction i...,Nile Wilson shares his story of mental health ...,,Documentary,0.600,
1274081,273967,MatchMakers,An exploration of matchmaking and the pursuit ...,,Documentary,0.600,
1274082,839744,Berlin: Hasenheide,Documentary about the social microcosm of Hase...,"berlin, germany, park",Documentary,0.600,
1274083,839745,Puncture,When one of his crew goes missing with a bag f...,,"Action, Thriller",0.600,


In [53]:
df['cast_crew'] = df['cast_crew'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x) # Convert list to string
df[['title','cast_crew']]

,title,cast_crew
0,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ken W..."
1,Interstellar,"Matthew McConaughey, Anne Hathaway, Michael Ca..."
2,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,Avatar,"Sam Worthington, Zoe Saldaña, Sigourney Weaver..."
4,The Avengers,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ..."
...,...,...
1274080,The Silent Battle: Mental Health & Addiction i...,
1274081,MatchMakers,
1274082,Berlin: Hasenheide,
1274083,Puncture,


In [55]:
df.reset_index().to_csv('movie_details.csv', index=False) # Save the updated DataFrame to a CSV file

In [ ]:
import pandas as pd

test = pd.read_csv('movie_details.csv')
test.head(10)
test['text'] 

,index,id,title,overview,keywords,genres,popularity,cast_crew
0,0,27205,Inception,"Cobb, a skilled thief who commits corporate es...","rescue, mission, dream, airplane, paris, franc...","Action, Science Fiction, Adventure",83.952,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ken W..."
1,1,157336,Interstellar,The adventures of a group of explorers who mak...,"rescue, future, spacecraft, race against time,...","Adventure, Drama, Science Fiction",140.241,"Matthew McConaughey, Anne Hathaway, Michael Ca..."
2,2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"joker, sadism, chaos, secret identity, crime f...","Drama, Action, Crime, Thriller",130.643,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","future, society, culture clash, space travel, ...","Action, Adventure, Fantasy, Science Fiction",79.932,"Sam Worthington, Zoe Saldaña, Sigourney Weaver..."
4,4,24428,The Avengers,When an unexpected enemy emerges and threatens...,"new york city, superhero, shield, based on com...","Science Fiction, Action, Adventure",98.082,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ..."
5,5,293660,Deadpool,The origin story of former Special Forces oper...,"superhero, anti hero, mercenary, based on comi...","Action, Adventure, Comedy",72.735,"Ryan Reynolds, Morena Baccarin, Ed Skrein, T.J..."
6,6,299536,Avengers: Infinity War,As the Avengers and their allies have continue...,"sacrifice, magic, superhero, based on comic, s...","Adventure, Action, Science Fiction",154.340,"Robert Downey Jr., Chris Evans, Chris Hemswort..."
7,7,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,"dual identity, rage and hate, based on novel o...",Drama,69.498,"Edward Norton, Brad Pitt, Helena Bonham Carter..."
8,8,118340,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...","spacecraft, based on comic, space, orphan, adv...","Action, Science Fiction, Adventure",33.255,"Chris Pratt, Zoe Saldaña, Dave Bautista, Vin D..."
9,9,680,Pulp Fiction,"A burger-loving hit man, his philosophical par...","drug dealer, boxer, massage, stolen money, bri...","Thriller, Crime",74.862,"John Travolta, Samuel L. Jackson, Uma Thurman,..."


In [ ]:
test.loc[test['index'].isin([1, 2, 3])]

,index,id,title,overview,keywords,genres,popularity,cast_crew
1,1,157336,Interstellar,The adventures of a group of explorers who mak...,"rescue, future, spacecraft, race against time,...","Adventure, Drama, Science Fiction",140.241,"Matthew McConaughey, Anne Hathaway, Michael Ca..."
2,2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"joker, sadism, chaos, secret identity, crime f...","Drama, Action, Crime, Thriller",130.643,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","future, society, culture clash, space travel, ...","Action, Adventure, Fantasy, Science Fiction",79.932,"Sam Worthington, Zoe Saldaña, Sigourney Weaver..."


In [28]:
import faiss
print(faiss.__version__)
print(faiss.get_compile_options())


1.12.0
OPTIMIZE AVX2 
